In [1]:
import pymysql
import pandas as pd
import numpy as np
#from sqlalchemy import create_engine

In [2]:
config_read = {'host':'mysql-rfam-public.ebi.ac.uk','port':4497,'user':'rfamro','db':'Rfam','charset':'utf8mb4'} #,'password':'none'
config_insert = {'host':'remotemysql.com','port':3306,'user':'nxG0uS1RMI','password':'UG9Opfcult','db':'nxG0uS1RMI','charset':'utf8mb4'} 

In [2]:
config_read = {'host':'34.84.45.183','port':3306,'user':'btorin','password':'77ssrdw@yb','db':'ssr_dw_yb01','charset':'utf8mb4'} #,'password':'none'


In [3]:
def read_mysql_table(sql_text,config_read):
    con=pymysql.connect(**config_read) #打开数据库连接
    df = pd.read_sql(sql_text, con)
    con.close()#关闭数据库连接
    return(df)

def insert_mysql_table(sql_text,val):
    con=pymysql.connect(**config_insert) #打开数据库连接
    cursor = con.cursor()
    sql_text = sql_text
    val=val
    try:
        cursor.executemany(sql_text,val) # 执行sql语句
        con.commit() # 提交到数据库执行
    except Exception as e:
        print(e)   # 如果发生错误则回滚
        con.rollback()
    con.close()#关闭数据库连接

In [11]:
sql_test = 'select user_id,username,created_at,last_login,balance,level_name from member_info where level_name != "測試帳號" and balance >= 1; '
data_test_table=read_mysql_table(sql_test,config_read)

In [12]:
sql_test_123 = 'select user_id,sum(deposit_count) as count from deposit_record group by user_id ; '
data_user_dep=read_mysql_table(sql_test_123,config_read)

In [21]:
final_data_user=pd.merge(data_test_table,data_user_dep,on = 'user_id',how = 'left')

In [23]:
final_data_user['count']=final_data_user['count'].fillna(0)

In [26]:
final_data_user=final_data_user[final_data_user['count']==0]

In [28]:
final_data_user=final_data_user[['username','created_at','last_login','balance','level_name']]

In [31]:
final_data_user.shape[0]

1367010

In [32]:
test12=final_data_user[0:700000]

In [34]:
test34=final_data_user[700000:1400000]

In [38]:
test12.to_csv('F:/Desktop/user_data/user_list1.csv' ,encoding="utf_8_sig" )

In [39]:
test34.to_csv('F:/Desktop/user_data/user_list2.csv' ,encoding="utf_8_sig" )

In [41]:
test34.tail()

,username,created_at,last_login,balance,level_name
1515577,17342051503,2021-01-12 02:49:32,2021-01-12 02:49:33,38.75,一般會員
1515578,qbj123456789,2021-01-12 02:49:41,2021-01-12 02:49:42,38.00,一般會員
1515579,a73848725,2021-01-12 02:50:03,2021-01-12 02:50:04,38.25,一般會員
1515580,hodxijgkgzggi,2021-01-12 02:50:10,2021-01-12 02:50:10,38.50,一般會員
1515581,159753xxx,2021-01-12 02:50:15,2021-01-12 02:50:15,39.50,一般會員


In [74]:
#成功  審核時間
data_user = pd.read_csv("F:/Downloads/daily_register (1).csv" , encoding = "utf-8")
data_user.shape
data_user=data_user.fillna(0)
data_user['注册时间']=data_user['注册时间'].astype(str).str.replace("=","").str.replace('"',"")
data_user['主帐户余额']=data_user['主帐户余额'].astype(str).str.replace("=","").str.replace('"',"")
data_user['损益']=data_user['损益'].astype(str).str.replace("=","").str.replace('"',"")

In [75]:
#批量寫入DB中   
sql_text_write="INSERT INTO user_table(ad, name_1, name_2,name_3, name_4, ip,time, country, city, balance,payoff, level, state_1, freez_1,stop_1,lock_1) \
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) \
                ON DUPLICATE KEY UPDATE  ad=VALUES(ad),name_1=VALUES(name_1),name_2=VALUES(name_2),name_3=VALUES(name_3),name_4=VALUES(name_4),ip=VALUES(ip),time=VALUES(time),country=VALUES(country),city=VALUES(city),balance=VALUES(balance),payoff=VALUES(payoff),level=VALUES(level),state_1=VALUES(state_1),freez_1=VALUES(freez_1),stop_1=VALUES(stop_1),lock_1=VALUES(lock_1)  ;"  
insert_mysql_table(sql_text_write,data_user.values.tolist())

In [76]:
sql_test = 'select * from user_table where name_1="gcg0603t" '
data_test_table=read_mysql_table(sql_test,config_insert)

In [77]:
data_test_table.head()

,ad,name_1,name_2,name_3,name_4,ip,time,country,city,balance,payoff,level,state_1,freez_1,stop_1,lock_1
0,1,gcg0603t,0,hall0,agent0,240e:370:3f26:2f80:3d00:dbb0:66df:6725,2020-06-01 00:00:01,中国,0,0.00,0.00,一般會員,已停用,未冻结,未停权,未锁定


In [63]:
data_test_table.iloc[0:1]["lock_1"] = '已解鎖'

In [70]:
insert_mysql_table(sql_text_write,data_test_table.values.tolist())

In [57]:
sql_text="select * from family;"  #SELECT * FROM Information_Schema.TABLES select * from family ;
rowdata=read_mysql_table(sql_text,config_read)
#rowdata.dtypes rowdata.memory_usage(index=False, deep=True).sum()

In [67]:
'''
#資料整理
data_nosie=rowdata[rowdata['noise_cutoff']>rowdata['noise_cutoff'].mean()]
data_auther=data_nosie[data_nosie['author']=='Griffiths-Jones SR']
data_time=data_auther[data_auther['created']>'2020-09-01 00:00:00']
'''

"\n#資料整理\ndata_nosie=rowdata[rowdata['noise_cutoff']>rowdata['noise_cutoff'].mean()]\ndata_auther=data_nosie[data_nosie['author']=='Griffiths-Jones SR']\ndata_time=data_auther[data_auther['created']>'2020-09-01 00:00:00']\n"

In [89]:
#批量寫入DB中   #ON DUPLICATE KEY UPDATE `c`=VALUES(`a`)+VALUES(`b`)
sql_text_write="INSERT INTO test_table(rfam_acc, rfam_id, auto_wiki) VALUES (%s,%s,%s) ;"  #SELECT * FROM Information_Schema.TABLES select * from family ;
insert_mysql_table(sql_text_write,save_data.values.tolist())

In [5]:
sql_test = 'select * from test_table'
data_test_table=read_mysql_table(sql_test,config_insert)

In [6]:
data_test_table

,rfam_acc,rfam_id,auto_wiki
0,RF00001,5S_rRNA,123
1,RF00002,5_8S_rRNA,123
2,RF00003,U1,123
3,RF00004,U2,123
4,RF00005,tRNA,123
...,...,...,...
3440,RF03551,mir-506,123
3441,RF03552,mir-43_2,123
3442,RF03553,mir-190_2,123
3443,RF03554,mir-92_2,123


In [86]:
con=pymysql.connect(**config_insert)
cursor = con.cursor() 
# 创建数据库的sql(如果数据库存在就不创建，防止异常)
# 创建表
sql_2 = '''CREATE TABLE `test_table` (
  `rfam_acc` CHAR(20) NOT NULL,
  `rfam_id` CHAR(50) NOT NULL,
  `auto_wiki` INT NOT NULL,
  PRIMARY KEY (`rfam_acc`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
'''
cursor.execute(sql_2)
con.close()

In [85]:
con=pymysql.connect(**config_insert)
cursor = con.cursor() 
# 创建数据库的sql(如果数据库存在就不创建，防止异常)
# 创建表
sql_2 = "DROP TABLE IF EXISTS {}".format('test_table')
cursor.execute(sql_2)
con.close()

In [186]:
con=pymysql.connect(**config_insert)
cursor = con.cursor() 
# 创建数据库的sql(如果数据库存在就不创建，防止异常)
# 创建表
sql_2 = '''CREATE TABLE `user_table` (
  `ad` int NOT NULL,
  `name_1` CHAR(20) ,
  `name_2` CHAR(20) ,
  `name_3` CHAR(20) ,
  `name_4` CHAR(20) ,
  `ip` CHAR(80) ,
  `time` CHAR(20) ,
  `country` CHAR(20) ,
  `city` CHAR(20) ,
  `balance` CHAR(20) ,
  `payoff` CHAR(20) ,
  `level` CHAR(20) ,
  `state_1` CHAR(20) ,
  `freez_1` CHAR(20) ,
  `stop_1` CHAR(20) ,
  `lock_1` CHAR(20) ,
  PRIMARY KEY (`ad`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
'''
cursor.execute(sql_2)
con.close()

In [185]:
con=pymysql.connect(**config_insert)
cursor = con.cursor() 
# 创建数据库的sql(如果数据库存在就不创建，防止异常)
# 创建表
sql_2 = "DROP TABLE IF EXISTS {}".format('user_table')
cursor.execute(sql_2)
con.close()

In [ ]:
con=pymysql.connect(**config_insert)
cursor = con.cursor() 
# 创建数据库的sql(如果数据库存在就不创建，防止异常)
# 创建表
sql_2 = "DROP TABLE IF EXISTS {}".format('accounts')
cursor.execute(sql_2)
cursor.close()
con.close()

In [30]:
con=pymysql.connect(**config_insert)
cursor = con.cursor() 
# 创建数据库的sql(如果数据库存在就不创建，防止异常)
# 创建表
sql_2 = '''CREATE TABLE IF NOT EXISTS `accounts` (
    `id` int(11) NOT NULL AUTO_INCREMENT,
    `username` varchar(50) NOT NULL,
    `password` varchar(255) NOT NULL,
    `email` varchar(100) NOT NULL,
    PRIMARY KEY (`id`)
) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;

'''
cursor.execute(sql_2)
con.close()

In [42]:
con=pymysql.connect(**config_insert)
cursor = con.cursor() 
# 创建数据库的sql(如果数据库存在就不创建，防止异常)
# 创建表
ttt="INSERT INTO accounts (username, password, email) VALUES ('torin','123qwe','yywer@gmail.com')" 
cursor.execute(ttt)
cursor.close()
con.close()

In [43]:
con=pymysql.connect(**config_insert) #打开数据库连接
df = pd.read_sql('select * from accounts ', con)
con.close()
df

,id,username,password,email
0,1,123qwe,123qwe,123qwe@gaskdask.com
1,4,test,test,test@gnail.com
